<a href="https://colab.research.google.com/github/Victordisa/Modelo_atalanta/blob/master/Modelo_atalanta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import  max
from pyspark.sql.functions import col, when
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyspark.sql.functions import corr
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pandas as pd
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [4]:
from google.colab import drive

In [5]:
drive.mount("/drive")

Mounted at /drive


In [6]:
spark = SparkSession.builder.appName("Datos_atalanta").getOrCreate()

In [7]:
df = pd.read_excel('/content/Atalanta.xlsx')
df.head()

,Jornada,Estacion,horario,Lugar,Ranking,Oponente,Formacion,Asistecia,Resultado,total_tiros_local,...,corner_visitante,faltas_local,faltas_visitante,fuera_local,fuera_visitante,Arbitro,Gol_local_primero,Gol_visitante_primero,Gol_local_segundo,Gol_visitante_segundo
0,1,"Wed Sep 30, 2020",20:45:00,A,(3.),Lazio (6.),3-4-2-1,1000,01:04:00,14.0,...,3.0,16.0,11.0,0.0,0.0,Fabrio Maresca,0,3,1,1
1,2,"Sat Sep 26, 2020",15:00:00,A,(2.),Torino (12.),3-4-1-2,700,02:04:00,7.0,...,6.0,11.0,12.0,2.0,2.0,Massimiliano Irrati,2,3,0,1
2,3,"Sun Oct 4, 2020",12:30:00,H,(2.),Cagliari Calcio (14.),3-4-1-2,1000,05:02:00,21.0,...,9.0,11.0,9.0,2.0,0.0,fabrizio Pasqua,4,1,1,1
3,4,"Sat Oct 17, 2020",15:00:00,A,(1.),Napoli (6.),3-4-1-2,1000,04:01:00,19.0,...,2.0,12.0,23.0,1.0,1.0,Marco Di bello,4,0,0,1
4,5,"Sat Oct 24, 2020",15:00:00,H,(4.),Sampdoria (10.),3-4-1-2,1000,01:03:00,20.0,...,1.0,12.0,19.0,1.0,0.0,Gianpaolo Calvarese,0,1,1,2


In [8]:
# Leer el archivo de Excel
xls = pd.ExcelFile('/content/Atalanta.xlsx')

# Leer todas las hojas en un diccionario
sheets = {sheet_name: pd.read_excel(xls, sheet_name) for sheet_name in xls.sheet_names}

# Lista para almacenar los DataFrames modificados
modified_dfs = []

# Iterar sobre las hojas y modificar cada DataFrame
for sheet_name, df in sheets.items():
    # Verificar si la columna 'Resultado' existe en la hoja
    if 'Resultado' in df.columns:
        # Modificar la columna 'Ranking_oponente'
        df['Ranking_oponente'] = df['Oponente'].str.extract(r'\((\d+\.\))')
        df['Ranking_oponente'] = df['Ranking_oponente'].str.replace(r'[()]', '', regex=True)

        # Modificar la columna 'Resultado'
        df['Resultado'] = df['Resultado'].astype(str)
        df['Resultado'] = df['Resultado'].str.replace(r':00$', '', regex=True)  # Eliminar ':00'
        df['Resultado'] = df['Resultado'].str.replace(r'\b0', '', regex=True)    # Eliminar ceros a la izquierda

    # Si la columna 'asistencia' ya existe, eliminarla antes de agregar
    if 'Asistencia' in df.columns:
        df = df.drop(columns=['Asistencia'])

    # Agregar el DataFrame modificado a la lista
    modified_dfs.append(df)

# Concatenar todos los DataFrames modificados en uno solo
df_atalanta = pd.concat(modified_dfs, ignore_index=True)

df_atalanta.head(100)




,Jornada,Estacion,horario,Lugar,Ranking,Oponente,Formacion,Asistecia,Resultado,total_tiros_local,...,faltas_local,faltas_visitante,fuera_local,fuera_visitante,Arbitro,Gol_local_primero,Gol_visitante_primero,Gol_local_segundo,Gol_visitante_segundo,Ranking_oponente
0,1,"Wed Sep 30, 2020",20:45:00,A,(3.),Lazio (6.),3-4-2-1,1000,1:4,14.0,...,16.0,11.0,0.0,0.0,Fabrio Maresca,0,3,1,1,6.
1,2,"Sat Sep 26, 2020",15:00:00,A,(2.),Torino (12.),3-4-1-2,700,2:4,7.0,...,11.0,12.0,2.0,2.0,Massimiliano Irrati,2,3,0,1,12.
2,3,"Sun Oct 4, 2020",12:30:00,H,(2.),Cagliari Calcio (14.),3-4-1-2,1000,5:2,21.0,...,11.0,9.0,2.0,0.0,fabrizio Pasqua,4,1,1,1,14.
3,4,"Sat Oct 17, 2020",15:00:00,A,(1.),Napoli (6.),3-4-1-2,1000,4:1,19.0,...,12.0,23.0,1.0,1.0,Marco Di bello,4,0,0,1,6.
4,5,"Sat Oct 24, 2020",15:00:00,H,(4.),Sampdoria (10.),3-4-1-2,1000,1:3,20.0,...,12.0,19.0,1.0,0.0,Gianpaolo Calvarese,0,1,1,2,10.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20,"Sat Jan 28, 2023",20:45:00,H,(6.),Sampdoria (19.),3-4-2-1,NaN,2:0,21.0,...,17.0,11.0,1.0,1.0,Daniele Doveri,1,0,1,0,19.
96,21,"Sat Feb 4, 2023",20:45:00,A,(4.),Sassuolo (15.),37379,NaN,1:0,18.0,...,12.0,13.0,0.0,2.0,Matteo Marcenaro,0,0,1,0,15.
97,22,"Sat Feb 11, 2023",20:45:00,A,(5.),Lazio (4.),37379,NaN,0:2,13.0,...,12.0,14.0,0.0,0.0,Daniele Orsato,0,1,0,1,4.
98,23,"Sun Feb 19, 2023",12:30:00,H,(3.),Lecce (15.),3-4-2-1,NaN,1:2,20.0,...,10.0,12.0,2.0,2.0,Matteo Piccinini,0,1,1,1,15.


In [9]:
df_atalanta = df_atalanta.apply(lambda x: x.fillna(0).astype(int) if x.dtype == 'float' else x)
df_atalanta.head()

,Jornada,Estacion,horario,Lugar,Ranking,Oponente,Formacion,Asistecia,Resultado,total_tiros_local,...,faltas_local,faltas_visitante,fuera_local,fuera_visitante,Arbitro,Gol_local_primero,Gol_visitante_primero,Gol_local_segundo,Gol_visitante_segundo,Ranking_oponente
0,1,"Wed Sep 30, 2020",20:45:00,A,(3.),Lazio (6.),3-4-2-1,1000,1:4,14,...,16,11,0,0,Fabrio Maresca,0,3,1,1,6.
1,2,"Sat Sep 26, 2020",15:00:00,A,(2.),Torino (12.),3-4-1-2,700,2:4,7,...,11,12,2,2,Massimiliano Irrati,2,3,0,1,12.
2,3,"Sun Oct 4, 2020",12:30:00,H,(2.),Cagliari Calcio (14.),3-4-1-2,1000,5:2,21,...,11,9,2,0,fabrizio Pasqua,4,1,1,1,14.
3,4,"Sat Oct 17, 2020",15:00:00,A,(1.),Napoli (6.),3-4-1-2,1000,4:1,19,...,12,23,1,1,Marco Di bello,4,0,0,1,6.
4,5,"Sat Oct 24, 2020",15:00:00,H,(4.),Sampdoria (10.),3-4-1-2,1000,1:3,20,...,12,19,1,0,Gianpaolo Calvarese,0,1,1,2,10.


In [10]:
df_atalanta['Oponente'] = df_atalanta['Oponente'].str.replace(r'\(\d+\.\)', '', regex=True)
df_atalanta.head()

,Jornada,Estacion,horario,Lugar,Ranking,Oponente,Formacion,Asistecia,Resultado,total_tiros_local,...,faltas_local,faltas_visitante,fuera_local,fuera_visitante,Arbitro,Gol_local_primero,Gol_visitante_primero,Gol_local_segundo,Gol_visitante_segundo,Ranking_oponente
0,1,"Wed Sep 30, 2020",20:45:00,A,(3.),Lazio,3-4-2-1,1000,1:4,14,...,16,11,0,0,Fabrio Maresca,0,3,1,1,6.
1,2,"Sat Sep 26, 2020",15:00:00,A,(2.),Torino,3-4-1-2,700,2:4,7,...,11,12,2,2,Massimiliano Irrati,2,3,0,1,12.
2,3,"Sun Oct 4, 2020",12:30:00,H,(2.),Cagliari Calcio,3-4-1-2,1000,5:2,21,...,11,9,2,0,fabrizio Pasqua,4,1,1,1,14.
3,4,"Sat Oct 17, 2020",15:00:00,A,(1.),Napoli,3-4-1-2,1000,4:1,19,...,12,23,1,1,Marco Di bello,4,0,0,1,6.
4,5,"Sat Oct 24, 2020",15:00:00,H,(4.),Sampdoria,3-4-1-2,1000,1:3,20,...,12,19,1,0,Gianpaolo Calvarese,0,1,1,2,10.


In [12]:
#cuantos goles hace en casa prmer tiempo
df_atalanta_local = df_atalanta[(df_atalanta['Lugar'] == 'H')]
prom_atalanta_primero = df_atalanta_local['Gol_local_primero'].mean()
print(prom_atalanta_primero)

0.8571428571428571


In [13]:
#cuantos goles  le hacen en casa prmer tiempo
df_atalanta_local = df_atalanta[(df_atalanta['Lugar'] == 'H')]
prom_atalanta_primero = df_atalanta_local['Gol_visitante_primero'].mean()
print(prom_atalanta_primero)

0.4523809523809524


In [14]:
#cuantos goles hace fuera primer tiempo
df_atalanta_visitante = df_atalanta[(df_atalanta['Lugar'] == 'A')]
prom_atalanta_primero = df_atalanta_visitante['Gol_visitante_primero'].mean()
print(prom_atalanta_primero)

0.7701149425287356


In [15]:
#cuantos goles le hacen fuera en el primero tiempo
df_atalanta_visitante = df_atalanta[(df_atalanta['Lugar'] == 'A')]
prom_atalanta_primero = df_atalanta_visitante['Gol_local_primero'].mean()
print(prom_atalanta_primero)

0.5172413793103449


In [16]:
#cuantos goles hace en casa segundo tiempo
df_atalanta_local = df_atalanta[(df_atalanta['Lugar'] == 'H')]
prom_atalanta_segundo = df_atalanta_local['Gol_local_segundo'].mean()
print(prom_atalanta_segundo)

1.3333333333333333


In [62]:
#cuantos goles le hacen en casa segundo tiempo
df_atalanta_local = df_atalanta[(df_atalanta['Lugar'] == 'H')]
prom_atalanta_segundo = df_atalanta_local['Gol_visitante_segundo'].mean()
print(prom_atalanta_segundo)

0.8214285714285714


In [17]:
#cuantos goles le hacen fuera segundo tiempo
df_atalanta_visitante = df_atalanta[(df_atalanta['Lugar'] == 'A')]
prom_atalanta_segundo = df_atalanta_visitante['Gol_local_segundo'].mean()
print(prom_atalanta_segundo)

0.6206896551724138


In [18]:
#cuantos goles hace fuera segundo tiempo
df_atalanta_visitante = df_atalanta[(df_atalanta['Lugar'] == 'A')]
prom_atalanta_segundo = df_atalanta_visitante['Gol_visitante_segundo'].mean()
print(prom_atalanta_segundo)

0.9655172413793104


In [19]:
faltas_por_arbitro = df_atalanta_local.groupby('Arbitro')['faltas_local'].mean()
print(faltas_por_arbitro.sort_values(ascending=False))

Arbitro
Michael Fabbri          15.500000
Daniele Doveri          15.000000
fabio Maresca           15.000000
Gianluca manganiello    15.000000
Valerio Marini          15.000000
Davide Massa            14.750000
Maurizio Mariani        14.250000
Marco Di Bello          14.142857
Simone Sozza            13.600000
Livio Marinelli         13.500000
Fabio Maresca           13.000000
Massimiliano Irrati     12.666667
Marco Guida             12.500000
Paride Tremolada        12.000000
Davide Ghersini         12.000000
Gianpaolo Calvarese     12.000000
Alessandro Prontera     12.000000
federico La Penna       12.000000
Marco Piccinini         11.666667
Rosario Abisso          11.200000
fabrizio Pasqua         11.000000
Alberto Santoro         11.000000
Federico Dionisi        11.000000
Daniele Chiffi          11.000000
matteo Marcenaro        11.000000
Juan Luca Sacchi        10.666667
Matteo Marcenaro        10.000000
Matteo Piccinini        10.000000
Gianluca Manganiello    10.000000
Gianlu

In [20]:
faltas_por_arbitro = df_atalanta_visitante.groupby('Arbitro')['faltas_visitante'].mean()
print(faltas_por_arbitro.sort_values(ascending=False))

Arbitro
federico Dionisi        23.500000
Gianluca Manganiello    21.000000
Federico Dionisi        21.000000
Marco Di bello          19.000000
Anotio Rapuano          18.000000
Davide Massa            17.000000
Simone Sozza            17.000000
Gianpaolo Calvarese     16.500000
Giovanni Ayroldi        16.500000
Marco Serra             16.000000
Antonio Rapuano         15.800000
Daniele Chiffi          15.333333
Maurizio Mariani        15.200000
Luca Pairetto           14.750000
Daniele Doveri          14.500000
Marco Piccinini         14.500000
Daniele Orsato          14.400000
Marco Guida             14.333333
Andrea Colombo          14.000000
Matteo Marchetti        14.000000
Gianluca Aureliano      13.500000
Matteo Marcenaro        13.000000
Alessandro Prontera     13.000000
paolo Valeri            13.000000
Luca Massimi            13.000000
Francesco Fourneau      13.000000
Antonio Giua            13.000000
Massimiliano Irrati     12.000000
Michael Fabbri          12.000000
fabio 

In [21]:
df_atalanta["Oponente"] = df_atalanta["Oponente"].str.strip()


In [22]:
Atalanta_vs_juventus = df_atalanta[(df_atalanta["Oponente"] == "Juventus")]
Atalanta_vs_juventus.head(15)


,Jornada,Estacion,horario,Lugar,Ranking,Oponente,Formacion,Asistecia,Resultado,total_tiros_local,...,faltas_local,faltas_visitante,fuera_local,fuera_visitante,Arbitro,Gol_local_primero,Gol_visitante_primero,Gol_local_segundo,Gol_visitante_segundo,Ranking_oponente
11,12,"Wed Dec 16, 2020",18:30:00,A,(8.),Juventus,3-4-2-1,x,1:1,12,...,14,18,2,0,Daniele Doveri,1,0,0,1,4.
30,31,"Sun Apr 18, 2021",15:00:00,H,(4.),Juventus,3-4-1-2,x,1:0,14,...,18,16,0,0,Daniele Orsato,0,0,1,0,3.
51,14,"Sat Nov 27, 2021",18:00:00,A,(4.),Juventus,3-4-2-1,NaN,0:1,15,...,13,20,1,0,Giovanni Ayroldi,0,1,0,0,6.
62,25,"Sun Feb 13, 2022",20:45:00,H,(5.),Juventus,3-4-2-1,NaN,1:1,21,...,13,9,0,3,Maurizio Mariani,0,0,1,1,4.
94,19,"Sun Jan 22, 2023",20:45:00,A,(5.),Juventus,3-4-2-1,NaN,3:3,15,...,9,6,2,0,Livio Marinelli,2,1,1,2,7.
109,34,"Sun May 7, 2023",12:30:00,H,(4.),Juventus,3-4-2-1,NaN,0:2,24,...,13,16,0,2,Daniele Doveri,0,0,0,2,7.
120,7,"Sun Oct 1, 2023",20:45:00,H,(4.),Juventus,3-4-2-1,NaN,0:0,15,...,13,7,0,1,Daniele Chiffi,0,0,0,0,3.
141,28,"Sun Mar 10, 2024",18:00:00,A,(6.),Juventus,3-4-2-1,NaN,2:2,18,...,12,10,2,0,Marco Guida,0,1,2,1,2.


In [24]:
atalanta_total = Atalanta_vs_juventus[Atalanta_vs_juventus["Lugar"] == "H"][["total_tiros_local", "total_tiros_visitante", "Tiros_puerta_local", "tiros_puerta_visitante",
                                                                                 "corner_local", "corner_visitante", "Gol_local_primero", "Gol_local_segundo", "Gol_visitante_primero",
                                                                                 "Gol_visitante_segundo", "faltas_local", "faltas_visitante"]].mean()
atalanta_total



,0
total_tiros_local,18.50
total_tiros_visitante,10.25
Tiros_puerta_local,3.00
tiros_puerta_visitante,1.50
corner_local,5.75
corner_visitante,5.25
Gol_local_primero,0.00
Gol_local_segundo,0.50
Gol_visitante_primero,0.00
Gol_visitante_segundo,0.75


In [26]:
def obtener_resultado(Resultado):
    goles_local, goles_visitante = map(int, Resultado.split(':'))  # Separar y convertir en enteros
    if goles_local > goles_visitante:
        return 'Victoria'
    elif goles_local < goles_visitante:
        return 'Derrota'
    else:
        return 'Empate'

In [27]:
# Aplicar la función para crear una nueva columna con los resultados
Atalanta_vs_juventus['resultado_partido'] = Atalanta_vs_juventus['Resultado'].apply(obtener_resultado)

# Contar las victorias, empates y derrotas
conteo_resultados = Atalanta_vs_juventus['resultado_partido'].value_counts()
conteo_resultados




<ipython-input-27-be1df3ea1846>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Atalanta_vs_juventus['resultado_partido'] = Atalanta_vs_juventus['Resultado'].apply(obtener_resultado)


,count
resultado_partido,
Empate,5
Derrota,2
Victoria,1


In [26]:
Atalanta_vs_udinese['Oponente'] = Atalanta_vs_udinese['Oponente'].astype('category').cat.codes

<ipython-input-26-38da58057d5f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Atalanta_vs_udinese['Oponente'] = Atalanta_vs_udinese['Oponente'].astype('category').cat.codes


In [28]:

X = Atalanta_vs_udinese[["Oponente", "corner_local", "corner_visitante", "posecion_local",
                         "Tiros_puerta_local", "tiros_puerta_visitante"]]

# Variables objetivo: Goles (local y visitante), corners (local y visitante), tiros a puerta (local y visitante)
y_goles_local = atalanta_total["Gol_local_primero"]
y_goles_visitante = atalanta_total["Gol_visitante_primero"]
y_goles_local = atalanta_total["Gol_local_segundo"]
y_goles_visitante = atalanta_total["Gol_visitante_segundo"]

y_corners_local = atalanta_total["corner_local"]
y_corners_visitante = atalanta_total["corner_visitante"]

y_tiros_puerta_local = atalanta_total["Tiros_puerta_local"]
y_tiros_puerta_visitante = atalanta_total["tiros_puerta_visitante"]


X_train, X_test, y_train_goles_local, y_test_goles_local = train_test_split(X, y_goles_local, test_size=0.2, random_state=42)
X_train, X_test, y_train_goles_visitante, y_test_goles_visitante = train_test_split(X, y_goles_visitante, test_size=0.2, random_state=42)

X_train, X_test, y_train_corners_local, y_test_corners_local = train_test_split(X, y_corners_local, test_size=0.2, random_state=42)
X_train, X_test, y_train_corners_visitante, y_test_corners_visitante = train_test_split(X, y_corners_visitante, test_size=0.2, random_state=42)

X_train, X_test, y_train_tiros_puerta_local, y_test_tiros_puerta_local = train_test_split(X, y_tiros_puerta_local, test_size=0.2, random_state=42)
X_train, X_test, y_train_tiros_puerta_visitante, y_test_tiros_puerta_visitante = train_test_split(X, y_tiros_puerta_visitante, test_size=0.2, random_state=42)

# Paso 3: Crear el modelo de regresión lineal para cada variable objetivo
model_goles_local = LinearRegression()
model_goles_local.fit(X_train, y_train_goles_local)

model_goles_visitante = LinearRegression()
model_goles_visitante.fit(X_train, y_train_goles_visitante)

model_corners_local = LinearRegression()
model_corners_local.fit(X_train, y_train_corners_local)

model_corners_visitante = LinearRegression()
model_corners_visitante.fit(X_train, y_train_corners_visitante)

model_tiros_puerta_local = LinearRegression()
model_tiros_puerta_local.fit(X_train, y_train_tiros_puerta_local)

model_tiros_puerta_visitante = LinearRegression()
model_tiros_puerta_visitante.fit(X_train, y_train_tiros_puerta_visitante)

# Paso 4: Hacer predicciones para el próximo partido
# Ejemplo de características para el próximo partido (estas deben estar basadas en los datos más recientes)
proximo_partido = pd.DataFrame({
    'Oponente': [1],  # Codificado como 1 para Udinese (o el valor correspondiente)
    'corner_local': [5],  # Ejemplo de características: Atalanta con 5 corners
    'corner_visitante': [3],  # Udinese con 3 corners
    'posecion_local': [55],  # Ejemplo de posesión de Atalanta (55%)
    'Tiros_puerta_local': [7],  # Ejemplo de tiros a puerta de Atalanta
    'tiros_puerta_visitante': [5]  # Ejemplo de tiros a puerta de Udinese
})

# Predicciones para el próximo partido
goles_local_pred = model_goles_local.predict(proximo_partido)
goles_visitante_pred = model_goles_visitante.predict(proximo_partido)

corners_local_pred = model_corners_local.predict(proximo_partido)
corners_visitante_pred = model_corners_visitante.predict(proximo_partido)

tiros_puerta_local_pred = model_tiros_puerta_local.predict(proximo_partido)
tiros_puerta_visitante_pred = model_tiros_puerta_visitante.predict(proximo_partido)

# Mostrar las predicciones
print(f"Predicción de goles para Atalanta (local): {goles_local_pred[0]}")
print(f"Predicción de goles para Udinese (visitante): {goles_visitante_pred[0]}")

print(f"Predicción de corners para Atalanta (local): {corners_local_pred[0]}")
print(f"Predicción de corners para Udinese (visitante): {corners_visitante_pred[0]}")

print(f"Predicción de tiros a puerta para Atalanta (local): {tiros_puerta_local_pred[0]}")
print(f"Predicción de tiros a puerta para Udinese (visitante): {tiros_puerta_visitante_pred[0]}")

TypeError: Input should have at least 1 dimension i.e. satisfy `len(x.shape) > 0`, got scalar `1.0` instead.